<a href="https://colab.research.google.com/github/markdevine/ET5003_SEM1_2021-2/blob/main/ET5003_KaggleCompetition_MarkDevine_19187289.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://drive.google.com/uc?export=view&id=1vK33e_EqaHgBHcbRV_m38hx6IkG0blK_" width="350"/>
</div> 

#**Artificial Intelligence - MSc**
##ET5003 - MACHINE LEARNING APPLICATIONS 

###Instructor: Enrique Naredo
###ET5003_KaggleCompetition

In [ ]:
#@title Current Date
Today = '2021-10-09' #@param {type:"date"}


In [ ]:
#@markdown ---
#@markdown ### Enter your details here:
Team_Number = "" #@param {type:"string"}
Student_ID&Name = "Mark Devine" #@param {type:"string"}
Student_ID&Name = "Mark Devine" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
Student_ID&Name = "" #@param {type:"string"}
#@markdown ---

In [ ]:
#@title Notebook information
Notebook_type = 'Etivity' #@param ["Example", "Lab", "Practice", "Etivity", "Assignment", "Exam"]
Version = Draft #@param ["Draft", "Final"] {type:"raw"}
Submission = False #@param {type:"boolean"}

# INTRODUCTION

Your introduction here.

***The goal is to use advanced Machine Learning methods to predict House price.***

## Imports

In [57]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [58]:
# expand the display of the notebook when run
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [59]:
# standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
# to plot
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D

# to generate classification, regression and clustering datasets
import sklearn.datasets as dt

# to create data frames
from pandas import DataFrame

# to generate data from an existing dataset
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [61]:
# Scikit-learn is an open source machine learning library 
# that supports supervised and unsupervised learning
# https://scikit-learn.org/stable/
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [62]:
# Regular expression operations
#https://docs.python.org/3/library/re.html
import re 

# Natural Language Toolkit
# https://www.nltk.org/install.html
import nltk

# Stemming maps different forms of the same word to a common “stem” 
# https://pypi.org/project/snowballstemmer/
from nltk.stem import SnowballStemmer

# https://www.nltk.org/book/ch02.html
from nltk.corpus import stopwords

# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler

In [63]:
!pip install gpy

In [64]:
import GPy as GPy
import numpy as np
import pylab as pb
import pymc3 as pm
import arviz as az

In [65]:
# Define the seed so that results can be reproduced
seed = 11
rand_state = 11

# Define the color maps for plots
color_map = plt.cm.get_cmap('RdYlBu')
color_map_discrete = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","cyan","magenta","blue"])

In [66]:
# display complete contents and information of a dataframe without any kind of truncation (https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# DATASET

Extract from this [paper](https://ieeexplore.ieee.org/document/9300074):

* House prices are a significant impression of the economy, and its value ranges are of great concerns for the clients and property dealers. 

* Housing price escalate every year that eventually reinforced the need of strategy or technique that could predict house prices in future. 

* There are certain factors that influence house prices including physical conditions, locations, number of bedrooms and others.


1. [Download the dataset](https://github.com/UL-CS6134/CS6134_SEM1_2021-2/tree/main/Week-5). 

2. Upload the dataset into your folder.



The challenge is to predict the final price of each house.

### Loading the housing price dataset

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
# Path, copy the path from your Drive
path = '/content/drive/MyDrive/Colab Notebooks/Data/Kaggle/'

# Housing Price Data
train_data = path + 'HousePrice_Train.csv'
test_data = path + 'HousePrice_Test.csv'

# train data
df_train = pd.read_csv(train_data)
# X_train_original = df_train.drop("label",axis=1).values
# y_train = df_train.label.values
print(df_train.shape)

# test data
df_test = pd.read_csv(test_data)
# X_test_original = df_test.drop("label",axis=1).values
# y_test = df_test.label.values
print(df_test.shape)

(1638, 14)
(702, 13)


## Training & Test Data

In [14]:
# split data into training and test
from sklearn.model_selection import train_test_split

# training: 70% (0.7), test: 30% (0.3) 
# you could try any other combination 
# but consider 50% of training as the low boundary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

NameError: ignored

### Train dataset

In [ ]:
# training dataset: 
training_file = syntPath+filename1
# test dataset: 
testing_file = syntPath+filename2
# cost dataset: 
cost_file = syntPath+filename3

In [69]:
# show first data frame rows 
df_train.head()

,Index,ID,Location,Num_Bathrooms,Num_Beds,BER_class,Description,Services,Features,Latitude,Longitude,Type,Surface,Price
0,1,12409116,Ongar,2,2,C3,"DNG are delighted to present to the market no. 5 Chipping Row. This two bedroom apartment comes to the market in excellent condition throughout. Viewers will be sure to see the meticulous care and attention the home owner has for this property. \n\r\nThis well-presented apartment comprises of an open plan living and dining room, kitchen, two spacious double bedrooms the master benefitting from the a large en-suite and an extensively tiled bathroom. This property benefits from own door access, designated car parking and a quiet and private location.\n\r\nChipping Row is well serviced by excellent public transport links to Dublin City centre. Dublin Airport, N3 / M3 and M50 are all within an easy reach. Local primary and secondary schools, Blanchardstown Shopping Centre are all within driving distance. Ongar village and all it has to offer including supermarket, restaurants, shops and pharmacy€TM is only a short walk away.\n\r\nViewing comes highly recommended.\n\r\nFeatures:\r\nGround floor two bedroom apartment\r\n67 sqm / 721 sqft\r\nBuilt c. 2005\r\nDouble glazed windows throughout\r\nElectric heating\r\nOwn door access\r\nDesignated and visitor car parking \r\nManagement fee €"" approx. €760\r\nManagement Company €"" CORE\n\n\r\nAccommodation\r\nLiving/dining room 5.2m x 3.3m\r\nLaminate flooring, feature fireplace, large window to the front of the property.\r\nKitchen 3.4m x 1\r\nLaminate flooring, wall and floor fitted kitchen units, plumbed for washing machine. \r\nBedroom 1 5.1m x 2.2m\r\nLarge double bedroom, fitted wardrobes, ample storage space, en-suite off.\r\nEn-suite 2.5m x 1.6m\r\nExtensively tiled, shower, whb, wc.\r\nBedroom 2 3.7m x 2.7m\r\nLarge double bedroom, fitted wardrobes, ample storage space.\r\nBathroom 2.5m x 1.7m\r\nExtensively tiled, bath with over-head shower, whb, wc.",NaN,None,53.396659,-6.438791,apartment,67.00,195000
1,2,12320330,North Strand,1,2,D1,"13 Northbrook Terrace is a charming red brick period home set in a secluded location within walking distance of Dublin's City Centre. Behind the red brick façade the light filled accommodation offers a contemporary feel while maintaining charming original features such as high ceilings, ceiling coving, beautiful cast iron fireplaces and antique style radiators. The welcoming entrance hallway is finished with beautiful American Oak flooring, a fan light above the door floods this space with natural light, this leads through to two spacious interconnecting reception rooms, from here the garden is accessed through French doors. The kitchen is located at the end of the hallway this space is fitted with modern high gloss wall and floor units there is plumbing for a dishwasher and washing machine. Ample dining space creates an ideal place to relax. A feature floor to ceiling window floods this space with natural light. French doors lead to the garden. Upstairs there are two generous double bedrooms both with feature fireplaces, sash windows and American Oak flooring. The bathroom on the return with feature roll top bath completes the accommodation on this level. The further appeal of this charming home is the converted attic space, flooded with natural light with the aid of sky lights, it offers an ideal space for a wide variety of uses. \n\n\rOutside the secluded south facing rear garden is low maintenance, its laid in gravel and features a raised patio area making this sun trap an ideal space to dine alfresco. The ideal location ensures this charming home is walking distance to Connolly Station for Dart &amp; Luas with the Cross-City Luas stop at Marlborough Street, IFSC and the Dublin Dockland's also accessible on foot. There are numerous shops, restaurants and schools in the immediate locality. The Mater hospital, O'Connell Street and city colleges are all easily accessible on foot. All in all, this is a

In [70]:
# Generate descriptive statistics
df_train.describe()

,Index,ID,Num_Bathrooms,Num_Beds,Latitude,Longitude,Surface,Price
count,1638.000000,1.638000e+03,1638.000000,1638.000000,1638.000000,1638.000000,1638.000000,1.638000e+03
mean,819.500000,1.231930e+07,2.054335,3.043346,53.350473,-6.255086,169.540695,5.547196e+05
std,472.994186,1.447540e+05,1.241259,1.379650,0.076832,0.144430,1791.793934,5.652032e+05
min,1.000000,1.118567e+07,1.000000,1.000000,52.501856,-6.497000,3.400000,1.999500e+04
25%,410.250000,1.228104e+07,1.000000,2.000000,53.294261,-6.309613,73.000000,2.950000e+05
50%,819.500000,1.238055e+07,2.000000,3.000000,53.341386,-6.247780,97.050000,3.950000e+05
75%,1228.750000,1.240548e+07,3.000000,4.000000,53.382031,-6.193982,135.000000,5.950000e+05
max,1638.000000,1.242836e+07,18.000000,27.000000,53.630588,-1.744995,72236.387140,8.900000e+06


### Test dataset

In [71]:
# show first data frame rows 
df_test.head()

,Index,ID,Location,Num_Bathrooms,Num_Beds,BER_class,Description,Services,Features,Latitude,Longitude,Type,Surface
0,1639,12292473,Milltown,2,2,C2,"OMD Estate Agents are delighted to present this 2 bed townhouse situated in this well sought after location. The property is presented in immaculate condition and is conveniently located within a short stroll of Milltown, Ranelagh and Donnybrook villages. \r\nThe accommodation comprises of 2 double bedrooms, both rooms with built in wardrobes and family bathroom. Downstairs consists of good sized living area with stove fire, under stairs storage room, kitchen/ dining room with all integrated goods and w.c. There is a sunny paved terrace to the rear.\n\r\nIt has O.F.C.H, PVC double glazing throughout, designated parking space. Local amenities include Luas, bus routes, shops, Wilde and Green, David Lloyd Riverview gym, restaurants, bars and U.C.D.\n\r\nLiving Room 4.46m x 3.56m\r\nKitchen 4.00m x 3.65m\r\nW.c 1.65m x 1.90m\r\nBed 1) 3.11m x 3.53m\r\nBed 2) 2.63m x 3.54m\r\nBathroom 2.68m x 1.43m\r\nLanding 2.57m x .82m","Parking,Oil Fired Central Heating","Designated parking spaces\nPresented in pristine condition\nPVC double glazing throughout\nO.F.C.H\nSunny terrace\nManagement fees €1,300 per annum.\n",53.316269,-6.243391,townhouse,65.0
1,1640,12314667,Glasnevin,2,5,E1,"Welcome to Saint Anthony No.214 Glasnevin Avenue, a substantial c.142sq.m 5 bedroom residence with the added benefit of a c.60 foot long rear garden with glass house. Beyond this attractive facade is a dream property for prospective buyers looking to put their own stamp on a home that requires some upgrading.\n\r\nO'Connor Estate Agents are honoured to present this fine family home to the market which will appeal to families looking for a strategically located residence on the ever popular tree lined Glasnevin Avenue. This home presents endless opportunities with excellent potential.\n\r\nAccommodation:\n\r\nPorch area: c.2.64m x .90 leading to main entrance door.\n\r\nHallway: with carpets and downstairs WC.\n\r\nLiving Area: c.3.73m x 3.59m with coving, carpets and feature fireplace with electric fire.\n\r\nDining Area: c.4.03m x 3.73m with coving, carpets and fireplace.\n\r\nReception: c.2.95m x 2.41m with leading to reception extension (c.3.64m x 2.73m) flooded with natural light throughout.\n\r\nKitchen: c.2.91m x 1.91m with built in kitchen units.\n\r\nGarage: c.5.02m x 2.83 with excellent potential to convert subject to PP.\n\r\nBedroom 1: c.4.14m x c.2.80m with original flooring and walk in wardrobes.\n\r\nBedroom 2: c.2.92m x 2.63m with carpets throughout.\n\r\nBedroom 3: c.4.08m x 3.74m with carpets and full length built in wardrobes.\n\r\nBedroom 4: c.3.73m x 3.68m with carpets throughout.\n\r\nBedroom 5: c.2.81m x 2.70m with original flooring and walk in wardrobes.\n\r\nBathroom: c.2.65m x 2.61m fully tiled with walk in shower, WC, WHB, recessed lighting and hotpress.\n\r\nOutside: The front of the property has off street parking for a number of cars which is finished with a lawn and shrubbery. The rear of the property has a c.60 foot long garden with glass house, an outbuilding and access to annex and outside WC.\n\r\n**TRANSPORT** This property has the convenience of the M50 &amp; M1 within close proximity which opens up the location to the entire county &amp; country. There are numerous bus routes on Glasnevin Avenue offering transport links to Dublin City Centre. Dublin Airport is also within close proximity.\n\r\n**AMENITIES** Johnstown Park a renowned 12.75 hectare park is minutes from this property which boasts mature planting and tree lined walks to include tennis courts, playing pitches for soccer &amp; GAA and a playground to mention a few. The Botanical Gardens is within striking distance which is a popular destination for local residents. Other parks in the vicinity include Hampstead Park and Griffith Park. The Mater, Rotunda, Temple Street and The Bons Secours Hospital are located within clo

In [72]:
# Generate descriptive statistics
df_test.describe()

,Index,ID,Num_Bathrooms,Num_Beds,Latitude,Longitude,Surface
count,702.000000,7.020000e+02,702.000000,702.000000,702.000000,702.000000,702.000000
mean,1989.500000,1.231545e+07,2.027066,2.987179,53.350445,-6.252411,132.603530
std,202.794231,1.420680e+05,1.071273,1.198480,0.073961,0.091600,330.347066
min,1639.000000,1.147889e+07,1.000000,1.000000,53.214227,-6.521183,32.100000
25%,1814.250000,1.227637e+07,1.000000,2.000000,53.295225,-6.304258,73.000000
50%,1989.500000,1.237928e+07,2.000000,3.000000,53.341153,-6.247799,99.200000
75%,2164.750000,1.240421e+07,3.000000,4.000000,53.381752,-6.184277,136.875000
max,2340.000000,1.242824e+07,9.000000,14.000000,53.617443,-6.057150,8576.000000


In [73]:
# calculate the number of unique values in each column of the dataframe
nunique = df_train.nunique()
print(nunique)

Index            1638
ID               1638
Location         142 
Num_Bathrooms    11  
Num_Beds         11  
BER_class        16  
Description      1638
Services         31  
Features         1118
Latitude         1604
Longitude        1605
Type             10  
Surface          666 
Price            267 
dtype: int64


There are no unique values for any of the features.

In [74]:
# let's check the missing values across the attributes as a percentage (https://medium.com/dunder-data/finding-the-percentage-of-missing-values-in-a-pandas-dataframe-a04fa00f84ab)
df_train.isna().mean().round(4) * 100

Index            0.00 
ID               0.00 
Location         0.00 
Num_Bathrooms    0.00 
Num_Beds         0.00 
BER_class        15.32
Description      0.00 
Services         64.77
Features         0.00 
Latitude         0.00 
Longitude        0.00 
Type             0.00 
Surface          0.00 
Price            0.00 
dtype: float64

The dataset is mostly complete so we replace the missing values in the BER_class feature with the median value. 

The fact that the Services feature has 65% missing values doesn't matter as I intend on combining the test features, Description, Services and Features for the NLP.

In [75]:
# let's check the BER_class values
print(df_train['BER_class'].sort_values().unique())

['A1' 'A2' 'A3' 'B1' 'B2' 'B3' 'C1' 'C2' 'C3' 'D1' 'D2' 'E1' 'E2' 'F' 'G'
 'SINo666of2006exempt' nan]


In [76]:
# let's check the BER_class values for NaNs
df_train.BER_class.drop_duplicates()

0      C3                 
1      D1                 
2      A3                 
3      E1                 
7      NaN                
10     G                  
11     E2                 
15     C2                 
16     SINo666of2006exempt
19     F                  
24     B2                 
27     C1                 
36     D2                 
63     B3                 
146    B1                 
184    A2                 
800    A1                 
Name: BER_class, dtype: object

In [79]:
# encode the ordinal BER_class
# create a mapper

scale_mapper = {
    'A1': 1, 
    'A2': 2, 
    'A3': 3, 
    'B1': 4, 
    'B2': 5, 
    'B3': 6, 
    'C1': 7, 
    'C2': 8, 
    'C3': 9, 
    'D1': 10, 
    'D2': 11, 
    'E1': 12, 
    'E2': 13, 
    'F': 14, 
    'G': 15,
    'SINo666of2006exempt': 16
}

df_train['BER_class'] = df_train['BER_class'].replace(scale_mapper)
df_test['BER_class'] = df_test['BER_class'].replace(scale_mapper)
df_train.head()

,Index,ID,Location,Num_Bathrooms,Num_Beds,BER_class,Description,Services,Features,Latitude,Longitude,Type,Surface,Price
0,1,12409116,Ongar,2,2,9.0,"DNG are delighted to present to the market no. 5 Chipping Row. This two bedroom apartment comes to the market in excellent condition throughout. Viewers will be sure to see the meticulous care and attention the home owner has for this property. \n\r\nThis well-presented apartment comprises of an open plan living and dining room, kitchen, two spacious double bedrooms the master benefitting from the a large en-suite and an extensively tiled bathroom. This property benefits from own door access, designated car parking and a quiet and private location.\n\r\nChipping Row is well serviced by excellent public transport links to Dublin City centre. Dublin Airport, N3 / M3 and M50 are all within an easy reach. Local primary and secondary schools, Blanchardstown Shopping Centre are all within driving distance. Ongar village and all it has to offer including supermarket, restaurants, shops and pharmacy€TM is only a short walk away.\n\r\nViewing comes highly recommended.\n\r\nFeatures:\r\nGround floor two bedroom apartment\r\n67 sqm / 721 sqft\r\nBuilt c. 2005\r\nDouble glazed windows throughout\r\nElectric heating\r\nOwn door access\r\nDesignated and visitor car parking \r\nManagement fee €"" approx. €760\r\nManagement Company €"" CORE\n\n\r\nAccommodation\r\nLiving/dining room 5.2m x 3.3m\r\nLaminate flooring, feature fireplace, large window to the front of the property.\r\nKitchen 3.4m x 1\r\nLaminate flooring, wall and floor fitted kitchen units, plumbed for washing machine. \r\nBedroom 1 5.1m x 2.2m\r\nLarge double bedroom, fitted wardrobes, ample storage space, en-suite off.\r\nEn-suite 2.5m x 1.6m\r\nExtensively tiled, shower, whb, wc.\r\nBedroom 2 3.7m x 2.7m\r\nLarge double bedroom, fitted wardrobes, ample storage space.\r\nBathroom 2.5m x 1.7m\r\nExtensively tiled, bath with over-head shower, whb, wc.",NaN,None,53.396659,-6.438791,apartment,67.00,195000
1,2,12320330,North Strand,1,2,10.0,"13 Northbrook Terrace is a charming red brick period home set in a secluded location within walking distance of Dublin's City Centre. Behind the red brick façade the light filled accommodation offers a contemporary feel while maintaining charming original features such as high ceilings, ceiling coving, beautiful cast iron fireplaces and antique style radiators. The welcoming entrance hallway is finished with beautiful American Oak flooring, a fan light above the door floods this space with natural light, this leads through to two spacious interconnecting reception rooms, from here the garden is accessed through French doors. The kitchen is located at the end of the hallway this space is fitted with modern high gloss wall and floor units there is plumbing for a dishwasher and washing machine. Ample dining space creates an ideal place to relax. A feature floor to ceiling window floods this space with natural light. French doors lead to the garden. Upstairs there are two generous double bedrooms both with feature fireplaces, sash windows and American Oak flooring. The bathroom on the return with feature roll top bath completes the accommodation on this level. The further appeal of this charming home is the converted attic space, flooded with natural light with the aid of sky lights, it offers an ideal space for a wide variety of uses. \n\n\rOutside the secluded south facing rear garden is low maintenance, its laid in gravel and features a raised patio area making this sun trap an ideal space to dine alfresco. The ideal location ensures this charming home is walking distance to Connolly Station for Dart &amp; Luas with the Cross-City Luas stop at Marlborough Street, IFSC and the Dublin Dockland's also accessible on foot. There are numerous shops, restaurants and schools in the immediate locality. The Mater hospital, O'Connell Street and city colleges are all easily accessible on foot. All in all, this i

In [81]:
# replace the nans with the median value of BER_class
df_train['BER_class'].fillna(df_train['BER_class'].median(), inplace=True)

In [82]:
# let's double check the BER_class values for NaNs
df_train.BER_class.drop_duplicates()

0      9.0 
1      10.0
2      3.0 
3      12.0
10     15.0
11     13.0
15     8.0 
16     16.0
19     14.0
24     5.0 
27     7.0 
36     11.0
63     6.0 
146    4.0 
184    2.0 
800    1.0 
Name: BER_class, dtype: float64

### Expected Cost dataset

Natural language processing ([NLP](https://en.wikipedia.org/wiki/Natural_language_processing)) is a subfield of linguistics, computer science, and artificial intelligence.

* NLP concerned with the interactions between computers and human language.
* In particular how to program computers to process and analyze large amounts of natural language data. 
* The goal is a computer capable of "understanding" the contents of documents.
* Including the contextual nuances of the language within them. 
* The technology can then accurately extract information and insights contained in the documents.
* As well as categorize and organize the documents themselves. 

# NATURAL LANGUAGE PROCESSING

Your comments, explanation, and references here.

In [83]:
# combine the text features, Description, Services and Features
df_train['Property_Description'] = df_train['Description'] + ' ' + df_train['Services'] + ' ' + df_train['Features']

In [84]:
# drop the text features, Description, Services and Features
df_train.drop(['Description', 'Services', 'Features'], axis = 1, inplace = True)

In [87]:
# show first data frame rows 
df_train.head(100)

,Index,ID,Location,Num_Bathrooms,Num_Beds,BER_class,Latitude,Longitude,Type,Surface,Price,Property_Description
0,1,12409116,Ongar,2,2,9.0,53.396659,-6.438791,apartment,67.00,195000,NaN
1,2,12320330,North Strand,1,2,10.0,53.357129,-6.239779,terraced,95.97,425000,NaN
2,3,12405953,Stepaside,3,3,3.0,53.266468,-6.222771,semi-detached,107.00,535000,NaN
3,4,12202582,Cabinteely,1,3,12.0,53.261600,-6.166738,detached,81.00,499000,NaN
4,5,12299336,Lusk,3,4,10.0,53.550731,-6.121415,detached,153.00,510000,NaN
5,6,12225979,Glasnevin,1,3,12.0,53.380367,-6.279310,terraced,140.00,360000,NaN
6,7,12209989,North Circular Road,1,2,9.0,53.355659,-6.294885,apartment,70.98,300000,NaN
7,8,12084555,Clondalkin,2,4,10.0,53.318857,-6.391826,detached,244.98,590000,NaN
8,9,12405966,Finglas,3,2,10.0,53.400071,-6.304156,terraced,72.00,229000,NaN
9,10,12202195,Howth,2,2,10.0,53.366407,-6.077866,semi-detached,63.70,499500,NaN


In [22]:
# show first data frame rows 
df_test.head()

,Location,Description,Services,Features,Type
0,Milltown,"OMD Estate Agents are delighted to present this 2 bed townhouse situated in this well sought after location. The property is presented in immaculate condition and is conveniently located within a short stroll of Milltown, Ranelagh and Donnybrook villages. \r\nThe accommodation comprises of 2 double bedrooms, both rooms with built in wardrobes and family bathroom. Downstairs consists of good sized living area with stove fire, under stairs storage room, kitchen/ dining room with all integrated goods and w.c. There is a sunny paved terrace to the rear.\n\r\nIt has O.F.C.H, PVC double glazing throughout, designated parking space. Local amenities include Luas, bus routes, shops, Wilde and Green, David Lloyd Riverview gym, restaurants, bars and U.C.D.\n\r\nLiving Room 4.46m x 3.56m\r\nKitchen 4.00m x 3.65m\r\nW.c 1.65m x 1.90m\r\nBed 1) 3.11m x 3.53m\r\nBed 2) 2.63m x 3.54m\r\nBathroom 2.68m x 1.43m\r\nLanding 2.57m x .82m","Parking,Oil Fired Central Heating","Designated parking spaces\nPresented in pristine condition\nPVC double glazing throughout\nO.F.C.H\nSunny terrace\nManagement fees €1,300 per annum.\n",townhouse
1,Glasnevin,"Welcome to Saint Anthony No.214 Glasnevin Avenue, a substantial c.142sq.m 5 bedroom residence with the added benefit of a c.60 foot long rear garden with glass house. Beyond this attractive facade is a dream property for prospective buyers looking to put their own stamp on a home that requires some upgrading.\n\r\nO'Connor Estate Agents are honoured to present this fine family home to the market which will appeal to families looking for a strategically located residence on the ever popular tree lined Glasnevin Avenue. This home presents endless opportunities with excellent potential.\n\r\nAccommodation:\n\r\nPorch area: c.2.64m x .90 leading to main entrance door.\n\r\nHallway: with carpets and downstairs WC.\n\r\nLiving Area: c.3.73m x 3.59m with coving, carpets and feature fireplace with electric fire.\n\r\nDining Area: c.4.03m x 3.73m with coving, carpets and fireplace.\n\r\nReception: c.2.95m x 2.41m with leading to reception extension (c.3.64m x 2.73m) flooded with natural light throughout.\n\r\nKitchen: c.2.91m x 1.91m with built in kitchen units.\n\r\nGarage: c.5.02m x 2.83 with excellent potential to convert subject to PP.\n\r\nBedroom 1: c.4.14m x c.2.80m with original flooring and walk in wardrobes.\n\r\nBedroom 2: c.2.92m x 2.63m with carpets throughout.\n\r\nBedroom 3: c.4.08m x 3.74m with carpets and full length built in wardrobes.\n\r\nBedroom 4: c.3.73m x 3.68m with carpets throughout.\n\r\nBedroom 5: c.2.81m x 2.70m with original flooring and walk in wardrobes.\n\r\nBathroom: c.2.65m x 2.61m fully tiled with walk in shower, WC, WHB, recessed lighting and hotpress.\n\r\nOutside: The front of the property has off street parking for a number of cars which is finished with a lawn and shrubbery. The rear of the property has a c.60 foot long garden with glass house, an outbuilding and access to annex and outside WC.\n\r\n**TRANSPORT** This property has the convenience of the M50 &amp; M1 within close proximity which opens up the location to the entire county &amp; country. There are numerous bus routes on Glasnevin Avenue offering transport links to Dublin City Centre. Dublin Airport is also within close proximity.\n\r\n**AMENITIES** Johnstown Park a renowned 12.75 hectare park is minutes from this property which boasts mature planting and tree lined walks to include tennis courts, playing pitches for soccer &amp; GAA and a playground to mention a few. The Botanical Gardens is within striking distance which is a popular destination for local residents. Other parks in the vicinity include Hampstead Park and Griffith Park. The Mater, Rotunda, Temple Street and The Bons Secours Hospital are located within close proximity and there are educational facilities in abundance such as St. Kevins not to mention DCU which is located nearby. Other exce

We will keep Location and Type features seperate for the moment and work on the other three text based features.


In [ ]:
# calculate the number of unique values in each column of the dataframe
nunique = df_train.nunique()
print(nunique)

In [25]:
# let's check the services values as there seems to be lots of NaNs
# df_train.Services.drop_duplicates()
# df_train.Description.drop_duplicates()
# df_train.Features.drop_duplicates()

0       None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
1       Bight and Spacious interior\nNumerous original features

# PIECEWISE REGRESSION

**Piecewise regression**, extract from [Wikipedia](https://en.wikipedia.org/wiki/Segmented_regression):

Segmented regression, also known as piecewise regression or broken-stick regression, is a method in regression analysis in which the independent variable is partitioned into intervals and a separate line segment is fit to each interval. 

* Segmented regression analysis can also be performed on 
multivariate data by partitioning the various independent variables. 
* Segmented regression is useful when the independent variables, clustered into different groups, exhibit different relationships between the variables in these regions. 

* The boundaries between the segments are breakpoints.

* Segmented linear regression is segmented regression whereby the relations in the intervals are obtained by linear regression. 

Your comments, explanation, and references here.

In [ ]:
# your code here

# BAYESIAN NN

A [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network) (also known as a Bayes network, Bayes net, belief network, or decision network) is a probabilistic graphical model that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

* Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. 
* For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. 
* Given symptoms, the network can be used to compute the probabilities of the presence of various diseases.

Your comments, explanation, and references here.

In [ ]:
# your code here

# SUMMARY

Your summary here.